In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [17]:
df = pd.read_csv("car_evaluation.csv")

In [139]:
df.head()

,vhigh,vhigh.1,2,2.1,small,low,unacc
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   vhigh    1727 non-null   object
 1   vhigh.1  1727 non-null   object
 2   2        1727 non-null   object
 3   2.1      1727 non-null   object
 4   small    1727 non-null   object
 5   low      1727 non-null   object
 6   unacc    1727 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [18]:
df.drop(["2","2.1"],axis=1)

,vhigh,vhigh.1,small,low,unacc
0,vhigh,vhigh,small,med,unacc
1,vhigh,vhigh,small,high,unacc
2,vhigh,vhigh,med,low,unacc
3,vhigh,vhigh,med,med,unacc
4,vhigh,vhigh,med,high,unacc
...,...,...,...,...,...
1722,low,low,med,med,good
1723,low,low,med,high,vgood
1724,low,low,big,low,unacc
1725,low,low,big,med,good


In [7]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.preprocessing import OneHotEncoder

In [19]:
x = df.iloc[:,:-1]  
y = df.iloc[:,-1]  

In [20]:
ohe = OneHotEncoder()

ohe.fit(x)
X_ohe = ohe.transform(x).toarray()

In [155]:
tree =  DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=0)

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_ohe, y, test_size = 0.33, random_state = 42)

In [158]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [159]:
y_pred = tree.predict(X_test)

In [161]:
from sklearn.metrics import accuracy_score

print('Model accuracy: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy: 0.8105


In [166]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[ 98,   0,  29,   0],
       [ 18,   0,   0,   0],
       [ 35,   0, 364,   0],
       [ 26,   0,   0,   0]], dtype=int64)

In [167]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         acc       0.55      0.77      0.64       127
        good       0.00      0.00      0.00        18
       unacc       0.93      0.91      0.92       399
       vgood       0.00      0.00      0.00        26

    accuracy                           0.81       570
   macro avg       0.37      0.42      0.39       570
weighted avg       0.77      0.81      0.79       570



In [168]:
from sklearn import model_selection
seed = 1
kfold = model_selection.KFold(n_splits=10, random_state=seed)
kfold 

KFold(n_splits=10, random_state=1, shuffle=False)

In [169]:
results = model_selection.cross_val_score(model, X_test, y_test, cv=kfold, scoring='accuracy')
print('Accuracy -val set: %.2f%% (%.2f)' % (results.mean()*100, results.std()))
results

Accuracy -val set: nan% (nan)


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

#### Random Forest Classifier

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

rfc = RandomForestClassifier(class_weight = "balanced")
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

In [14]:
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
cm

              precision    recall  f1-score   support

         acc       0.92      0.94      0.93       127
        good       0.73      0.61      0.67        18
       unacc       0.99      0.99      0.99       399
       vgood       0.79      0.88      0.84        26

    accuracy                           0.96       570
   macro avg       0.86      0.86      0.86       570
weighted avg       0.96      0.96      0.96       570



array([[120,   4,   2,   1],
       [  2,  11,   0,   5],
       [  5,   0, 394,   0],
       [  3,   0,   0,  23]], dtype=int64)

#### LSTM

In [15]:
import keras
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Dropout, LSTM
from keras.optimizers import Adam

Using TensorFlow backend.


In [16]:
df = pd.read_csv("car_evaluation.csv")
df.drop(["2","2.1"],axis=1)

In [31]:
#ohe.fit(y)
#Y_ohe = ohe.transform(y).toarray()
#X_train, X_test, y_train, y_test = train_test_split(X_ohe, Y_ohe, test_size = 0.33, random_state = 42)

In [32]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [22]:
classifier = Sequential()

In [33]:
classifier.add(CuDNNLSTM(128, input_shape=(X_train.shape[1:]), return_sequences=True))
classifier.add(Dropout(0.2))

ValueError: Input 0 is incompatible with layer cu_dnnlstm_2: expected ndim=3, found ndim=2